In [2]:
# load environment variables from .env file
from dotenv import load_dotenv
load_dotenv(override=True)

False

# Database

In [3]:
from backend.database.embedding import MockEmbedding, OpenAIEmbedding
from backend.database.vector_db import VectorDB, EmbedData, Field, EmbedField

In [11]:
class Dialog(EmbedData):
    speaker: str
    content: str = EmbedField()

class Meeting(EmbedData):
    timestamp: int # unix timestamp, serves as id
    
    # AI-generated
    title: str # shown on the sidebar
    summary: str # shown at the beginning of after-meeting 
    
    
class LawArticle(EmbedData):
    book: str
    article_number: str
    title: str
    content: str = EmbedField()
    
    
class CaseRecord(EmbedData):
    timestamp: int # unix timestamp
    title: str
    content: str = EmbedField()
    


## Creating a mocked database

In [6]:
case_id = 'JUSTICIUS-MARCO'
meeting_timestamp = 123

# dialogs
dialogs = [
    Dialog(speaker="Justicius", content="Hello, I'm Justicius, nice to meet you!"),
    Dialog(speaker="Marco", content="Hey, I'm Marco, nice to meet you!"),
    Dialog(speaker="Justicius", content="What can I help you with?"),
    Dialog(speaker="Marco", content="I'm looking for a lawyer."),
]

dialog_db = VectorDB(f"Dialog:{case_id}_{meeting_timestamp}", Dialog)
dialog_db.reset() # reset if exists
dialog_db.add(dialogs)

In [8]:
# test search

# load it again, just for testing
results = []
for meeting_timestamp in [123, 456]:
    dialog_db = VectorDB(f"Dialog:{case_id}_{meeting_timestamp}", Dialog)
    results(dialog_db.query(top=3, content="greeting"))

[(0.8165385768610883,
  Dialog(speaker='Marco', content="Hey, I'm Marco, nice to meet you!")),
 (0.8074972214012732,
  Dialog(speaker='Justicius', content="Hello, I'm Justicius, nice to meet you!")),
 (0.7728671743877149,
  Dialog(speaker='Justicius', content='What can I help you with?'))]

In [17]:
import json
with open('BJNR001950896_output.json', 'r') as f:
    BGB = json.load(f)
    BGB = BGB[:20]
    BGB = [
        LawArticle(
            book=article['jurabk'],
            article_number=article['enbez'],
            title=article['titel'],
            content=article['content']
        )
        for article in BGB
        if article['jurabk'] == 'BGB'
        and article.get('enbez')
        and article.get('titel')
        and article.get('content')
    ]

bgb_db = VectorDB("BGB", LawArticle)
bgb_db.reset()
bgb_db.add(BGB)

In [18]:
for article in bgb_db:
    print(article)

book='BGB' article_number='§ 2' title='Eintritt der Volljährigkeit' content='Die Volljährigkeit tritt mit der Vollendung des 18. Lebensjahres ein.'
book='BGB' article_number='§ 8' title='Wohnsitz nicht voll Geschäftsfähiger' content='Wer geschäftsunfähig oder in der Geschäftsfähigkeit beschränkt ist, kann ohne den Willen seines gesetzlichen Vertreters einen Wohnsitz weder begründen noch aufheben.'
book='BGB' article_number='§ 13' title='Verbraucher' content='Verbraucher ist jede natürliche Person, die ein Rechtsgeschäft zu Zwecken abschließt, die überwiegend weder ihrer gewerblichen noch ihrer selbständigen beruflichen Tätigkeit zugerechnet werden können.'
book='BGB' article_number='§ 11' title='Wohnsitz des Kindes' content='Ein minderjähriges Kind teilt den Wohnsitz der Eltern; es teilt nicht den Wohnsitz eines Elternteils, dem das Recht fehlt, für die Person des Kindes zu sorgen. Steht keinem Elternteil das Recht zu, für die Person des Kindes zu sorgen, so teilt das Kind den Wohnsitz

In [19]:
bgb_db.query(top=3, content="adult")

[(0.779483108544475,
  LawArticle(book='BGB', article_number='§ 2', title='Eintritt der Volljährigkeit', content='Die Volljährigkeit tritt mit der Vollendung des 18. Lebensjahres ein.')),
 (0.7533051767858291,
  LawArticle(book='BGB', article_number='§ 13', title='Verbraucher', content='Verbraucher ist jede natürliche Person, die ein Rechtsgeschäft zu Zwecken abschließt, die überwiegend weder ihrer gewerblichen noch ihrer selbständigen beruflichen Tätigkeit zugerechnet werden können.')),
 (0.7516114171899585,
  LawArticle(book='BGB', article_number='§ 11', title='Wohnsitz des Kindes', content='Ein minderjähriges Kind teilt den Wohnsitz der Eltern; es teilt nicht den Wohnsitz eines Elternteils, dem das Recht fehlt, für die Person des Kindes zu sorgen. Steht keinem Elternteil das Recht zu, für die Person des Kindes zu sorgen, so teilt das Kind den Wohnsitz desjenigen, dem dieses Recht zusteht. Das Kind behält den Wohnsitz, bis es ihn rechtsgültig aufhebt.'))]

# AI Assistant

In [1]:
from gpt_wrapper.tools import Tools, Toolkit, ToolList, function_tool

In [ ]:
class CaseDBToolkit(Toolkit):
    def __init__(self, case_id: str):
        self.all_meetings = [...]
        self.meeting_dbs = [VectorDB(f"meeting_db:{meeting_id}", Meeting) for meeting_id in self.all_meetings]
    
    @function_tool
    def add_meeting(self, timestamp: int, transcript: str):
        # use GPT endpoint to summarize the transcript and generate a short title
        content = gpt(f"Write a 1-paragraph summary for the following meeting transcript:\n{transcript}")
        title = gpt(f"Write a short title for the following meeting transcript:\n{transcript}")

        # add to database
        self.meeting_db.add(Meeting(
            date=timestamp,
            title=title,
            summary=content,
            full_content=transcript
        ))
    
    @function_tool
    def get_relevant_meeting_snippets(self, meeting_id: str, query: str):
        return self.meeting_dbs[meeting_id].search(query)